In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/inaugural_speeches.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# làm sạch và chuẩn hóa văn bản
df.loc[0, 'text'][:100]

In [ ]:
df['clean_text'] = df['text'].str.replace('[^a-zA-Z]', ' ')

In [ ]:
df['clean_text'] = df['clean_text'].str.lower()

In [ ]:
df.loc[0, 'clean_text'][:100]

In [ ]:
df['clean_text'] = df['clean_text'].str.replace('\s+', ' ')

In [ ]:
df.loc[0, 'clean_text'][:100]

In [ ]:
df['char_cnt'] = df['clean_text'].str.len()
df['word_cnt'] = df['clean_text'].str.split().str.len()
df['avg_word_length'] = df['char_cnt']/df['word_cnt']

In [ ]:
df[['Name','clean_text','char_cnt','word_cnt','avg_word_length']].head()

In [ ]:
# cho biết bài diễn văn nào có nhiều từ nhất?
df[df['word_cnt']==df['word_cnt'].max()]

In [ ]:
# cho biết bài diễn văn nào có ít từ nhất?
df[df['word_cnt']==df['word_cnt'].min()]

In [ ]:
df.dtypes

In [ ]:
# từ cột Date --> rút trích được năm
df['year'] = df['Date'].str.extract('(\d{4})')
df[['Name','clean_text','char_cnt','word_cnt','avg_word_length','Date','year']].head()

In [ ]:
df[df['year'].isnull()]

In [ ]:
df.loc[df['year'].isnull(), 'year'] = '1997'

In [ ]:
df[df['year'].isnull()]

## CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
text1 = 'Tôi đi học'
text2 = 'Tôi học trong trường tôi'
text3 = 'Trường tôi rất đẹp'
doc = [text1, text2, text3]
df_ = pd.DataFrame({'text':doc})
df_

In [ ]:
cv.fit(df_['text']) # Tokenization

In [ ]:
cv.get_feature_names_out()

In [ ]:
cv_transformed = cv.transform(df_['text']) # ma trận thưa thớt ở dạng lưu trữ

In [ ]:
cv_transformed

In [ ]:
cv_transformed.toarray()

In [ ]:
df_transformed = pd.DataFrame(cv_transformed.toarray(), columns=cv.get_feature_names_out())
df_transformed # Bag of Words

In [ ]:
df_ = pd.concat([df_, df_transformed], axis=1)
df_

### Áp dụng CountVectorizer cho cột clean_text

In [ ]:
cv = CountVectorizer()

In [ ]:
cv.fit(df['clean_text'])

In [ ]:
cv.get_feature_names_out()[:10]

In [ ]:
cv.get_feature_names_out()[-10:]

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
cv_transformed = cv.transform(df['clean_text'])
df_transformed = pd.DataFrame(cv_transformed.toarray(), columns=cv.get_feature_names_out()).add_prefix('Count_')
df_transformed.head()                  

In [ ]:
cv = CountVectorizer(min_df=0.2, max_df=0.8)

In [ ]:
cv_transformed = cv.fit_transform(df['clean_text'])
# fit: "học"
# transform: "áp dụng"
# kết quả ==> bag of words

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
cv.get_feature_names_out()[:20]

In [ ]:
cv = CountVectorizer(min_df=0.2, max_df=0.8, stop_words='english')

In [ ]:
cv_transformed = cv.fit_transform(df['clean_text'])
len(cv.get_feature_names_out())

## TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv = TfidfVectorizer(max_features=100, stop_words='english')

In [ ]:
cv_transformed = cv.fit_transform(df['clean_text'])
len(cv.get_feature_names_out())

In [ ]:
cv.get_feature_names_out()[:20]

In [ ]:
cv_transformed.toarray()[:5, :10]

In [ ]:
df_transformed = pd.DataFrame(cv_transformed.toarray(),
                             columns=cv.get_feature_names_out()).add_prefix('Count_')
df_transformed.head()

In [ ]:
df_transformed.iloc[:5, :5]

In [ ]:
# xét văn bản index=0, cho biết 10 từ có trọng số lớn nhất
df_transformed.iloc[0,:].sort_values(ascending=False).head(10)

In [ ]:
# cho biết 10 văn bản có từ "government" có trọng số cao nhất
ser = df_transformed['Count_government'].sort_values(ascending=False).head(10)
ser

In [ ]:
df[df.index.isin(ser.index)]

In [ ]:
# cho biết 10 từ có tổng trọng số cao nhất trong tất cả văn bản
df_transformed.sum().sort_values(ascending=False).head(10)